# **xarray-leaflet: big geo-data visualization in the Jupyter Notebook**

<img src="xarray.png" alt="drawing" width="300"/> <img src="leaflet.png" alt="drawing" width="300"/> <img src="jupyter.png" alt="drawing" width="100"/>

## https://github.com/davidbrochart/xarray_leaflet

## David Brochart

<img src="quantstack.svg" alt="drawing" width="130"/>

## QuantStack

In [ ]:
import matplotlib.pyplot as plt
import os
import requests
from tqdm import tqdm
import zipfile
import rioxarray
import numpy as np

# dataset: river flow accumulation for South America (USGS - Hydrosheds)

In [ ]:
url = 'https://edcintl.cr.usgs.gov/downloads/sciweb1/shared/hydrosheds/sa_30s_zip_grid/sa_acc_30s_grid.zip'
filename = os.path.basename(url)
name = filename[:filename.find('_grid')]
adffile = os.path.join(name, name, 'w001001.adf')

if not os.path.exists(adffile):
    r = requests.get(url, stream=True)
    with open(filename, 'wb') as f:
        total_length = int(r.headers.get('content-length'))
        for chunk in tqdm(r.iter_content(chunk_size=1024), total=(total_length/1024) + 1):
            if chunk:
                f.write(chunk)
                f.flush()
    zip = zipfile.ZipFile(filename)
    zip.extractall('.')

In [ ]:
da = rioxarray.open_rasterio(adffile, masked=True)
da

# matplotlib

In [ ]:
da = da.rio.write_nodata(np.nan)
da = da.sel(band=1)
da1 = np.sqrt(da)
da1.plot.imshow()

## ipympl: interactive matplotlib

In [ ]:
%matplotlib widget

In [ ]:
da1.plot.imshow()

# xarray-leaflet

In [ ]:
from xarray_leaflet.transform import passthrough, normalize
from ipyleaflet import Map, basemaps
from rasterio.warp import Resampling
import warnings
import xarray as xr
import scipy.ndimage

## Data pipeline: transformation functions

In [ ]:
transform0 = normalize
transform1 = passthrough

In [ ]:
def transform2(array, *args, **kwargs):
    tile_width = kwargs['tile_width']
    tile_height = kwargs['tile_height']
    ny, nx = array.shape
    wx = nx // (tile_width // 2)
    wy = ny // (tile_height // 2)
    dim = {}
    if wx > 1:
        dim['x'] = wx
    if wy > 1:
        dim['y'] = wy
    array = array.coarsen(**dim, boundary='pad')
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        array = xr.core.rolling.DataArrayCoarsen.max(array)
    return array

In [ ]:
def transform3(array):
    radius = 2
    circle = np.zeros((2*radius+1, 2*radius+1)).astype('uint8')
    y, x = np.ogrid[-radius:radius+1,-radius:radius+1]
    index = x**2 + y**2 <= radius**2
    circle[index] = 1
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        array = np.sqrt(array)
    array = scipy.ndimage.maximum_filter(array, footprint=circle)
    return array

In [ ]:
m = Map(center=[-20, -60], zoom=3, basemap=basemaps.CartoDB.DarkMatter, interpolation='nearest')
m

In [ ]:
l = da.leaflet.plot(m,
                    transform0=transform0,
                    transform1=transform1,
                    transform2=transform2,
                    transform3=transform3,
                    resampling=Resampling.max)
l.interact(opacity=(0.0, 1.0))

# Dynamic map

In [ ]:
transform0 = passthrough
transform1 = normalize

In [ ]:
m = Map(center=[-20, -60], zoom=3, basemap=basemaps.CartoDB.DarkMatter, interpolation='nearest')
m

In [ ]:
l = da.leaflet.plot(m,
                    transform0=transform0,
                    transform1=transform1,
                    transform2=transform2,
                    transform3=transform3,
                    dynamic=True,
                    resampling=Resampling.max)
l.interact(opacity=(0.0,1.0,0.1))

# https://github.com/davidbrochart/xarray_leaflet